# Obatining the Full Dataset of UserMirrorer

In our `UserMirrorer` dataset, the raw data from `MIND` and `MovieLens-1M` datasets are distributed under restrictive licenses and cannot
be included directly.

Therefore, this notebook provides a comprehensive, step-by-step pipeline to load the original archives, execute all necessary preprocessing
operations, and assemble the final UserMirrorer training, and test splits.

To derive the full dataset, just click "run all" to execute all cells.

------

In [ ]:
!git clone https://github.com/UserMirrorer/UserMirrorer

In [ ]:
%cd UserMirrorer
!pip install -U datasets tqdm uszipcode sqlalchemy-mate==2.0.0.0

In [ ]:
! wget https://recodatasets.z20.web.core.windows.net/newsrec/MINDlarge_train.zip
! wget https://recodatasets.z20.web.core.windows.net/newsrec/MINDlarge_dev.zip
! wget https://recodatasets.z20.web.core.windows.net/newsrec/MINDlarge_test.zip
! wget --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-1m.zip

In [ ]:
!unzip MINDlarge_train.zip -d MINDlarge
!unzip MINDlarge_dev.zip -d MINDlarge_dev
!unzip MINDlarge_test.zip -d MINDlarge_test
!mv MINDlarge_dev/behaviors.tsv MINDlarge/behaviors_valid.tsv
!mv MINDlarge_dev/news.tsv MINDlarge/news_valid.tsv
!mv MINDlarge_test/news.tsv MINDlarge/news_test.tsv
!unzip ml-1m.zip

In [ ]:
!python preprocessing/DataProcessor_ML-1M.py --source_path ml-1m --project_path UserM

In [ ]:
!python preprocessing/DataProcessor_MIND.py --source_path MINDlarge --project_path UserM

In [ ]:
import pandas as pd
from usermirrorer.strategy.mind_strategy import MINDMappingStrategy, MINDDataStrategy
from usermirrorer.strategy.ml1m_strategy import ML1MDataStrategy
from usermirrorer.formatter.mapping import MappingStrategy
from usermirrorer.formatter.formatter import DataFormatter
from usermirrorer.generator.template import texts_to_messages, convert_action_list

import random
import numpy as np

random.seed(0)
np.random.rand(0)


## Create Full Training Set and Eval Set

In [ ]:
from datasets import load_dataset

dataset = load_dataset("MirrorUser/UserMirrorer", split="train")
train = dataset.to_pandas()

dataset = load_dataset("MirrorUser/UserMirrorer-eval", split="test")
test = dataset.to_pandas()

### Movielens-1M

In [ ]:
data_formatter = DataFormatter(
    ds=ML1MDataStrategy("UserM", "ml-1m"),
    mp=MappingStrategy()
)

train_split = train[train["dataset"] == "ml-1m"].copy()
train_split["user_id"] = train_split["user_id"].astype(int)
train_split["item_id"] = train_split["item_id"].astype(int)
train_split["impression_list"] = train_split["impression_list"].apply(lambda x: [int(i) for i in x])

train_result = data_formatter.get_all_details(train_split)
train_result["prompt"] = train_result["text"].apply(lambda x: texts_to_messages(convert_action_list(x)))

train.loc[train_result.index, "prompt"] = train_result["prompt"]
train.loc[train_result.index, "messages_chosen"] = train.loc[train_result.index].apply(lambda x: x["prompt"] + [x["messages_chosen"][-1]], axis=1)
train.loc[train_result.index, "messages_rejected"] = train.loc[train_result.index].apply(lambda x: x["prompt"] + [x["messages_rejected"][-1]], axis=1)

test_split = test[test["dataset"] == "ml-1m"].copy()
test_split["user_id"] = test_split["user_id"].astype(int)
test_split["item_id"] = test_split["item_id"].astype(int)
test_split["impression_list"] = test_split["impression_list"].apply(lambda x: [int(i) for i in x])

test_result = data_formatter.get_all_details(test_split)
test.loc[test_result.index, "text"] = test_result["text"]

### MIND

In [ ]:
data_formatter = DataFormatter(
    ds=MINDDataStrategy("UserM", "MIND"),
    mp=MINDMappingStrategy()
)

train_split = train[train["dataset"] == "MIND"].copy()

train_result = data_formatter.get_all_details(train_split)
train_result["prompt"] = train_result["text"].apply(lambda x: texts_to_messages(convert_action_list(x)))

train.loc[train_result.index, "prompt"] = train_result["prompt"]
train.loc[train_result.index, "messages_chosen"] = train.loc[train_result.index].apply(lambda x: x["prompt"] + [x["messages_chosen"][-1]], axis=1)
train.loc[train_result.index, "messages_rejected"] = train.loc[train_result.index].apply(lambda x: x["prompt"] + [x["messages_rejected"][-1]], axis=1)

test_split = test[test["dataset"] == "MIND"].copy()

test_result = data_formatter.get_all_details(test_split)
test.loc[test_result.index, "text"] = test_result["text"]

In [ ]:
train = train.loc[:, ["dataset", "messages_chosen", "messages_rejected"]]
train.to_json("UserMirrorer-Full.jsonl", orient="records", lines=True)
train.to_json("UserMirrorer-Full.jsonl.gz", orient="records", lines=True, compression="gzip")

test = test.drop(columns=["impression_list"])
test.to_json("UserMirrorer-eval-Full.jsonl", orient="records", lines=True)
test.to_json("UserMirrorer-eval-Full.jsonl.gz", orient="records", lines=True, compression="gzip")


In [ ]:
!sha1sum UserMirrorer-eval-Full.jsonl
!sha1sum UserMirrorer-Full.jsonl

The SHA256 of the derived dataset file should be:

- `UserMirrorer-Full.jsonl`: `c3b819d7acab4d9b10dbafc411b18f49c5d17798`

- `UserMirrorer-eval-Full.jsonl`: `61e40dd5cb4649b6fd19c02afa7b13e4ec5ff276`

In [ ]:
from google.colab import files
files.download("UserMirrorer-Full.jsonl.gz")

In [ ]:
files.download("UserMirrorer-eval-Full.jsonl.gz")

The dataset file `UserMirrorer-Full.jsonl.gz` and `UserMirrorer-eval-Full.jsonl.gz` will be downloaded automatically. Or you can doanload it manually in *files*.